In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight
        
    Returns
    -------
    List with 'n' random recommendations where probability correspondes to weight of item_id
    """
    recs = items_weights.sample(n=5, 
                     replace=False,
                     weights=items_weights['weight']
                     )['item_id'].to_list()
    
    return recs

In [5]:
%%time

result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

items_weights = np.log(data_train.groupby('item_id').agg({'sales_value':sum}) + np.e).reset_index()
items_weights.columns = ['item_id', 'weight']
items_weights['weight'] = items_weights['weight'] / np.sum(items_weights['weight'])

result['weighted_random_recommendation'] = result['user_id'].map(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

Wall time: 7.22 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5661978, 1104938, 7466189, 1006760, 976715]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[835029, 10203593, 9924934, 825895, 896369]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [20]:
%%time
result = pd.read_csv('predictions_basic.csv')

# Converting string values to lists
result['actual'] = result['actual'].map(lambda str_lst: [int(id) for id in str_lst[1:-1].split()])
result['random_recommendation'] = result['random_recommendation'].map(lambda str_lst: [int(id) for id in str_lst[1:-1].split(',')])
result['popular_recommendation'] = result['popular_recommendation'].map(lambda str_lst: [int(id) for id in str_lst[1:-1].split(',')])
result['itemitem'] = result['itemitem'].map(lambda str_lst: [int(id) for id in str_lst[1:-1].split(',')])
result['cosine'] = result['cosine'].map(lambda str_lst: [int(id) for id in str_lst[1:-1].split(',')])
result['tfidf'] = result['tfidf'].map(lambda str_lst: [int(id) for id in str_lst[1:-1].split(',')])
result['own_purchases'] = result['own_purchases'].map(lambda str_lst: [int(id) for id in str_lst[1:-1].split(',')])

# weighted_random_recommendation Calulation
result['weighted_random_recommendation'] = result['user_id'].map(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)

Wall time: 6.34 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[9337581, 1070538, 819474, 1025924, 13416579]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[13038767, 12301323, 822985, 945633, 826146]"


In [21]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    return flags.sum() / len(recommended_list)

In [16]:
def precision_at_k_series(df, rec_list_col, bought_list_col, k=5):
    df = df[[rec_list_col, bought_list_col]].copy()
    df['zip'] = list(zip(df[rec_list_col], df[bought_list_col]))
    df['p@k'] = df['zip'].map(lambda zp: precision_at_k(recommended_list=list(zp[0]), bought_list=list(zp[1]), k=k))
    
    return df['p@k']

In [22]:
result['p@5 random_recommendation'] = precision_at_k_series(df=result,
                                                            rec_list_col='random_recommendation',
                                                            bought_list_col='actual',
                                                            k=5)
result['p@5 popular_recommendation'] = precision_at_k_series(df=result,
                                                             rec_list_col='popular_recommendation',
                                                             bought_list_col='actual',
                                                             k=5)

result['p@5 itemitem'] = precision_at_k_series(df=result,
                                               rec_list_col='itemitem',
                                               bought_list_col='actual',
                                               k=5)

result['p@5 cosine'] = precision_at_k_series(df=result,
                                             rec_list_col='cosine',
                                             bought_list_col='actual',
                                             k=5)

result['p@5 own_purchases'] = precision_at_k_series(df=result,
                                                    rec_list_col='own_purchases',
                                                    bought_list_col='actual',
                                                    k=5)

result['p@5 weighted_random_recommendation'] = precision_at_k_series(df=result,
                                                                     rec_list_col='weighted_random_recommendation',
                                                                     bought_list_col='actual',
                                                                     k=5)

In [25]:
# metris
result[['p@5 random_recommendation', 'p@5 popular_recommendation', 'p@5 itemitem', 'p@5 cosine', 'p@5 own_purchases', 'p@5 weighted_random_recommendation']].mean()

p@5 random_recommendation             0.000588
p@5 popular_recommendation            0.155240
p@5 itemitem                          0.033595
p@5 cosine                            0.035260
p@5 own_purchases                     0.179987
p@5 weighted_random_recommendation    0.001763
dtype: float64

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# в работе